# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from ham_to_sparse_matrix import *
from constants import *
import scipy.sparse.linalg

In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(4*aLat) + aLat*(qnP1-qnM1)*V(qn)/(4*aLat) )
fermionNI=sp.expand( -(xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} - \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) - \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [5]:
N=4
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [6]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} - \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) - \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

lam=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    return lam*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
ham

(-4*\chi^{\dagger}_{0}*\chi_{0} - 2*\chi^{\dagger}_{0}*\chi_{1} + 2*\chi^{\dagger}_{0}*\chi_{3} - 2*\chi^{\dagger}_{1}*\chi_{0} + 4*\chi^{\dagger}_{1}*\chi_{1} - 2*\chi^{\dagger}_{1}*\chi_{2} - 2*\chi^{\dagger}_{2}*\chi_{1} - 4*\chi^{\dagger}_{2}*\chi_{2} - 2*\chi^{\dagger}_{2}*\chi_{3} + 2*\chi^{\dagger}_{3}*\chi_{0} - 2*\chi^{\dagger}_{3}*\chi_{2} + 4*\chi^{\dagger}_{3}*\chi_{3} + 2*p_{0}**2 + 2*p_{1}**2 + 2*p_{2}**2 + 2*p_{3}**2 - q_{0}*q_{2} + 3*q_{0}**2 - q_{1}*q_{3} + 3*q_{1}**2 - q_{2}*q_{0} + 3*q_{2}**2 - q_{3}*q_{1} + 3*q_{3}**2)/4

In [8]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

-\chi^{\dagger}_{0}*\chi_{0} - \chi^{\dagger}_{0}*\chi_{1}/2 + \chi^{\dagger}_{0}*\chi_{3}/2 - \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{1} - \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{1}/2 - \chi^{\dagger}_{2}*\chi_{2} - \chi^{\dagger}_{2}*\chi_{3}/2 + \chi^{\dagger}_{3}*\chi_{0}/2 - \chi^{\dagger}_{3}*\chi_{2}/2 + \chi^{\dagger}_{3}*\chi_{3} - a^{\dagger}_{0}*a^{\dagger}_{2}/8 + 5*a^{\dagger}_{0}*a_{0}/8 - a^{\dagger}_{0}*a_{2}/8 + a^{\dagger}_{0}**2/8 - a^{\dagger}_{1}*a^{\dagger}_{3}/8 + 5*a^{\dagger}_{1}*a_{1}/8 - a^{\dagger}_{1}*a_{3}/8 + a^{\dagger}_{1}**2/8 - a^{\dagger}_{2}*a^{\dagger}_{0}/8 - a^{\dagger}_{2}*a_{0}/8 + 5*a^{\dagger}_{2}*a_{2}/8 + a^{\dagger}_{2}**2/8 - a^{\dagger}_{3}*a^{\dagger}_{1}/8 - a^{\dagger}_{3}*a_{1}/8 + 5*a^{\dagger}_{3}*a_{3}/8 + a^{\dagger}_{3}**2/8 + 5*a_{0}*a^{\dagger}_{0}/8 - a_{0}*a^{\dagger}_{2}/8 - a_{0}*a_{2}/8 + a_{0}**2/8 + 5*a_{1}*a^{\dagger}_{1}/8 - a_{1}*a^{\dagger}_{3}/8 - a_{1}*a_{3}/8 + a_{1}**2/8 - a_{2}

In [12]:
for cutoff in [2,3,4,5,6]:
    t=Timer('toHam')
    t.start()
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    t.stop()
    
    t=Timer('evals')
    t.start()
    ens=scipy.sparse.linalg.eigs(hamMat,k=10,which='SM')[0]
    t.stop()
    
    print("Lambda = ",cutoff)
    print(ens)
    print()

toHam timer took: 0.8126 seconds
evals timer took: 0.0192 seconds
Lambda =  2
[-0.4142144 -3.3017498e-09j  0.08578692-4.3490207e-09j
  0.08578694+1.1190963e-08j -0.4142143 +2.7779713e-08j
  0.08578651+8.9504217e-09j  0.08578669-5.5975655e-09j
  0.085787  +2.2005807e-08j  0.08578672+1.2214064e-08j
  0.08578652-3.6973580e-09j  0.08578666+1.9866945e-08j]

toHam timer took: 4.0475 seconds
evals timer took: 0.0196 seconds
Lambda =  3
[-0.10832835+4.3919246e-09j  0.5291899 +1.7036768e-08j
  0.6137356 +5.3477809e-09j  0.5291885 -2.0304995e-08j
  0.6137336 -1.4949892e-08j  0.9185449 +4.8629016e-09j
  0.9185444 +3.3793242e-08j  1.0748116 -3.6805896e-09j
  1.0811466 +2.3894950e-08j  1.0748017 +1.4962398e-08j]

toHam timer took: 22.9774 seconds
evals timer took: 0.0336 seconds
Lambda =  4
[-0.00145815-6.9414230e-09j  0.8285686 -7.5220470e-09j
  1.0798564 +7.7359390e-09j  0.8285714 +1.4340473e-08j
  1.0798575 -1.8423693e-08j  1.1880286 +2.3239249e-08j
  1.1880233 -1.3375924e-08j  1.3857905 -5.8249

In [10]:
N

4

In [10]:
for cutoff in [7,8]:
    t=Timer('toHam')
    t.start()
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    t.stop()
    
    t=Timer('evals')
    t.start()
    ens=scipy.sparse.linalg.eigs(hamMat,k=10,sigma=0)[0]
    t.stop()
    
    print("Lambda = ",cutoff)
    print(ens)
    print()

MemoryError: Unable to allocate 11.0 GiB for an array with shape (38416, 38416) and data type float64